<a href="https://colab.research.google.com/github/EkaterinaLap/-/blob/main/%D1%82%D0%BE%D0%BD%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D1%8C_%D0%BE%D1%82%D0%B7%D1%8B%D0%B2%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Загружаем необходимые библиотеки и модули для анализа тональности текста
import re
import torch
from transformers import AutoModelForSequenceClassification, BertTokenizerFast
import pandas as pd
from nltk.corpus import stopwords
from nltk import download
# Загружаем набор стоп-слов
download('stopwords')
# Выполняем предварительную обработку текста для анализа
def preprocess_text(text):
    text = re.sub(r'[^а-яА-ЯёЁa-zA-Z0-9\s]', '', text)
    text = text.lower()
    stop_words = set(stopwords.words('russian'))
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)
# Читаем отзывы из текстового файла
def read_reviews_from_txt(filename):
    with open(filename, "r", encoding="utf-8") as file:
        reviews = file.readlines()
    return [review.strip() for review in reviews]
# Загрузка модели и токенизатора для русского языка
model_name = "blanchefort/rubert-base-cased-sentiment"
tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
# Прогнозируем тональность текста
def predict_sentiment(text):
    text = preprocess_text(text)
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=1).squeeze()
    predicted_index = torch.argmax(probabilities).item()
    return probabilities.tolist(), predicted_index
# Читаем отзывы из текстового файла и сохраняем их в переменной
reviews = read_reviews_from_txt("texts.txt")
# Создание пустого списка для результатов
results = []
for review in reviews:
    probabilities, predicted_index = predict_sentiment(review)
    results.append({
        "content": review,
        "grade3": {0: "Нейтральная", 1: "Позитивная", 2: "Негативная"}[predicted_index],
        "prediction": predicted_index
    })
# Преобразование списка результатов в датафрейм
df = pd.DataFrame(results)
# Вывод датафрейма на экран
print(df)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                              content       grade3  prediction
0   'требую продолжения хочу ещё хочу 3 4 5 литера...  Нейтральная           0
1   'часа красивых картинок дени вильнева часто уп...  Нейтральная           0
2   'дюна картинок звука мало скажу сразу книгу чи...  Нейтральная           0
3   'кропотливая работа матерей кончиках пальцев н...   Позитивная           1
4   'рейтинг кинопоиске эээ это сравнению самобытн...   Негативная           2
5   'пророк дюна часть вторая самых ожидаемых филь...   Позитивная           1
6   'красивая обертка стоит признать люблю вильнёв...   Позитивная           1
7   'желание отомстить приводит пола настоящей свя...  Нейтральная           0
8   'красивый видеоряд сюжет вторая часть разорван...  Нейтральная           0
9   'вышли кинотеатра фильм огонь часа выпали реал...  Нейтральная           0
10  'дюна одна самых переоцененных книг средневеко...  Нейтральная           0
11  'фильма читал трилогию пытался посмотреть филь..